# GPU vs CPU ConvNet

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import time

## Get Data

In [2]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
train_dataloader = DataLoader(training_data, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=128, shuffle=True)

## Create Model

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(256, 120) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Train

In [4]:
def train(model,gpu=True,learning_rate=0.01,num_epochs = 10):
    if gpu:model = model.cuda()
    else:model = model.cpu()
    learning_rate=0.01
    num_epochs = 10
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        for inputs, labels in train_dataloader:
            if gpu:inputs,labels = inputs.cuda(),labels.cuda()
            else:inputs.cpu(),labels.cpu()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

## Test

In [5]:
def test(model):
    model = model.cpu()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct/total

## Test GPU vs CPU

In [7]:
models = [ConvNet(),ConvNet()]

for x in range(len(models)):
    start = time.time()
    train(models[x],gpu=x)
    acc = test(models[x])
    print(f'device = {["CPU","GPU"][x]} ; time is {time.time()-start} ; acc is {acc}')

device = CPU ; time is 26.932759523391724 ; acc is 0.9658
device = GPU ; time is 17.201897144317627 ; acc is 0.967
